In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
tet = test.Applicant_ID


In [4]:
train = pd.read_csv(r"C:\Users\King alagbe\Documents\Trainn.csv")
test = pd.read_csv(r"C:\Users\King alagbe\Documents\Testt.csv")
sample = pd.read_csv(r"C:\Users\King alagbe\Documents\Samplesubmission.csv")

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
test['form_field47'] = labelencoder.fit_transform(test['form_field47'])
train['form_field47'] = labelencoder.fit_transform(train['form_field47'])
train['default_status'] = labelencoder.fit_transform(train['default_status'])

In [7]:
train = train.round(3)
test = test.round(3)


In [8]:
c = train.drop(['Applicant_ID'], axis= 1)
c

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,3436.0,0.285,1.656,0.000,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.393,2.02,0.712,0.0,0.0,0,NaN,1.130,0.044,0
1,3456.0,0.674,0.234,0.000,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314,8.08,0.184,NaN,0.0,0,349.806,1.620,0.322,0
2,3276.0,0.538,3.151,0.000,6.282,NaN,956940.0,NaN,192944.0,1079864.0,...,0.163,18.18,0.791,0.0,0.0,0,NaN,1.513,0.012,1
3,3372.0,0.170,0.505,0.000,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,0.489,2.02,0.685,NaN,0.0,0,89.940,0.664,0.083,0
4,3370.0,0.773,1.101,0.000,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.275,12.12,0.438,0.0,0.0,0,97.888,1.428,0.046,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,3740.0,0.017,0.000,0.000,0.000,770998.0,9637475.0,4047934.0,11641992.0,19910965.0,...,0.000,9.09,0.730,NaN,0.0,1,591.049,0.000,0.242,0
55996,3360.0,2.011,0.625,0.000,0.000,NaN,927765.0,NaN,NaN,1849306.0,...,0.298,30.30,0.498,NaN,0.0,1,NaN,0.000,NaN,1
55997,3500.0,0.766,0.000,0.000,0.000,118645.0,3662435.0,3662435.0,3585024.0,704090.0,...,0.550,3.03,0.451,0.0,0.0,1,1252.474,2.000,0.163,0
55998,3280.0,0.052,2.092,2.221,0.000,NaN,3458599.0,NaN,115533.0,3458599.0,...,0.943,0.00,0.705,1.0,0.0,0,NaN,0.000,0.003,0


In [9]:
b =test.drop(['Applicant_ID'], axis  = 1)

In [10]:
train['total_severity'] = train['form_field3']-train['form_field4']
test['total_severity'] = test['form_field3']-test['form_field4']
train['form_field47_grouped'] = train.groupby('form_field47')['form_field14'].transform("mean")
test['form_field47_grouped'] = test.groupby('form_field47')['form_field14'].transform("mean")
train['form_field1'] = train['form_field1'].fillna(train['form_field1'].mean())
test['form_field1'] = test['form_field1'].fillna(test['form_field1'].mean())
train['diff_field8_field6'] = train['form_field8']-train['form_field6']
test['diff_field8_field6'] = test['form_field8']-test['form_field6']
train['diff_field50_field44'] = train['form_field50']-train['form_field44']
test['diff_field50_field44'] = test['form_field50']-test['form_field44']

In [11]:

train = c.fillna(-999, axis =1)
test = b.fillna(-999, axis =1)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_trans = sc.fit_transform(train.drop(['default_status'], axis = 1))
test_trans = sc.transform(test)

In [13]:
y = train.default_status
X = train_trans

In [15]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
stf = StratifiedKFold(n_splits=19, random_state=42, shuffle = True)
stf.get_n_splits(X,y)
test_scores = np.zeros(len(test_trans))
scores = []
for train, test in stf.split(X,y):
    x_train, x_test = X[train], X[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    cat = CatBoostClassifier(n_estimators=5000,reg_lambda=25)
    cat.fit(X,y,eval_set=[(x_train,y_train), (x_test,y_test)],early_stopping_rounds= 500, metric_period = 1000)
    scores.append(accuracy_score(y_test,cat.predict(x_test)))
    test_scores += cat.predict_proba(test_trans)[:,1]
    

0:	learn: 0.6738912	test: 0.6739670	test1: 0.6725272	best: 0.6725272 (0)	total: 70.5ms	remaining: 5m 52s
1000:	learn: 0.3744377	test: 0.3756114	test1: 0.3533167	best: 0.3533167 (1000)	total: 1m	remaining: 4m 1s
2000:	learn: 0.3495167	test: 0.3506424	test1: 0.3292579	best: 0.3292579 (2000)	total: 1m 59s	remaining: 2m 58s
3000:	learn: 0.3290208	test: 0.3300906	test1: 0.3097699	best: 0.3097699 (3000)	total: 2m 58s	remaining: 1m 58s
4000:	learn: 0.3094288	test: 0.3105037	test1: 0.2900845	best: 0.2900845 (4000)	total: 3m 58s	remaining: 59.4s
4999:	learn: 0.2918380	test: 0.2928432	test1: 0.2737471	best: 0.2737471 (4999)	total: 4m 57s	remaining: 0us

bestTest = 0.2737471109
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738993	test1: 0.6737453	best: 0.6737453 (0)	total: 77.9ms	remaining: 6m 29s
1000:	learn: 0.3744377	test: 0.3746019	test1: 0.3714823	best: 0.3714823 (1000)	total: 1m 1s	remaining: 4m 3s
2000:	learn: 0.3495167	test: 0.3497150	test1: 0.3459473	best: 0.3459473 (2000)	total: 2m	remaining: 3m 1s
3000:	learn: 0.3290208	test: 0.3292201	test1: 0.3254358	best: 0.3254358 (3000)	total: 3m	remaining: 2m
4000:	learn: 0.3094288	test: 0.3096508	test1: 0.3054329	best: 0.3054329 (4000)	total: 4m 1s	remaining: 1m
4999:	learn: 0.2918380	test: 0.2920499	test1: 0.2880238	best: 0.2880238 (4999)	total: 5m 2s	remaining: 0us

bestTest = 0.2880237852
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738894	test1: 0.6739229	best: 0.6739229 (0)	total: 76.4ms	remaining: 6m 21s
1000:	learn: 0.3744377	test: 0.3746196	test1: 0.3711645	best: 0.3711645 (1000)	total: 1m	remaining: 4m 1s
2000:	learn: 0.3495167	test: 0.3496626	test1: 0.3468910	best: 0.3468910 (2000)	total: 1m 59s	remaining: 2m 59s
3000:	learn: 0.3290208	test: 0.3291506	test1: 0.3266861	best: 0.3266861 (3000)	total: 2m 59s	remaining: 1m 59s
4000:	learn: 0.3094288	test: 0.3095496	test1: 0.3072534	best: 0.3072534 (4000)	total: 4m	remaining: 60s
4999:	learn: 0.2918380	test: 0.2919648	test1: 0.2895556	best: 0.2895556 (4999)	total: 5m	remaining: 0us

bestTest = 0.2895556295
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738824	test1: 0.6740497	best: 0.6740497 (0)	total: 79.7ms	remaining: 6m 38s
1000:	learn: 0.3744377	test: 0.3742037	test1: 0.3786490	best: 0.3786490 (1000)	total: 1m	remaining: 4m 2s
2000:	learn: 0.3495167	test: 0.3492972	test1: 0.3534657	best: 0.3534657 (2000)	total: 2m	remaining: 3m
3000:	learn: 0.3290208	test: 0.3288472	test1: 0.3321451	best: 0.3321451 (3000)	total: 3m	remaining: 2m
4000:	learn: 0.3094288	test: 0.3092490	test1: 0.3126643	best: 0.3126643 (4000)	total: 4m 1s	remaining: 1m
4999:	learn: 0.2918380	test: 0.2916676	test1: 0.2949029	best: 0.2949029 (4999)	total: 5m 2s	remaining: 0us

bestTest = 0.2949028537
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738824	test1: 0.6740498	best: 0.6740498 (0)	total: 79.7ms	remaining: 6m 38s
1000:	learn: 0.3744377	test: 0.3734816	test1: 0.3916434	best: 0.3916434 (1000)	total: 1m	remaining: 4m 3s
2000:	learn: 0.3495167	test: 0.3486587	test1: 0.3649563	best: 0.3649563 (2000)	total: 2m	remaining: 3m
3000:	learn: 0.3290208	test: 0.3282002	test1: 0.3437889	best: 0.3437889 (3000)	total: 3m	remaining: 2m
4000:	learn: 0.3094288	test: 0.3086747	test1: 0.3229994	best: 0.3229994 (4000)	total: 3m 57s	remaining: 59.4s
4999:	learn: 0.2918380	test: 0.2911471	test1: 0.3042712	best: 0.3042712 (4999)	total: 4m 53s	remaining: 0us

bestTest = 0.3042712136
bestIteration = 4999



...

0:	learn: 0.6738912	test: 0.6738849	test1: 0.6740041	best: 0.6740041 (0)	total: 99.2ms	remaining: 8m 15s
1000:	learn: 0.3744377	test: 0.3750781	test1: 0.3629099	best: 0.3629099 (1000)	total: 1m	remaining: 4m 1s
2000:	learn: 0.3495167	test: 0.3500852	test1: 0.3392825	best: 0.3392825 (2000)	total: 1m 57s	remaining: 2m 56s
3000:	learn: 0.3290208	test: 0.3295167	test1: 0.3200949	best: 0.3200949 (3000)	total: 2m 55s	remaining: 1m 56s
4000:	learn: 0.3094288	test: 0.3098677	test1: 0.3015270	best: 0.3015270 (4000)	total: 3m 56s	remaining: 59.1s
4999:	learn: 0.2918380	test: 0.2922478	test1: 0.2844606	best: 0.2844606 (4999)	total: 4m 54s	remaining: 0us

bestTest = 0.2844605674
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738947	test1: 0.6738275	best: 0.6738275 (0)	total: 73.8ms	remaining: 6m 9s
1000:	learn: 0.3744377	test: 0.3750184	test1: 0.3639832	best: 0.3639832 (1000)	total: 57.7s	remaining: 3m 50s
2000:	learn: 0.3495167	test: 0.3501497	test1: 0.3381217	best: 0.3381217 (2000)	total: 1m 54s	remaining: 2m 52s
3000:	learn: 0.3290208	test: 0.3296972	test1: 0.3168455	best: 0.3168455 (3000)	total: 2m 53s	remaining: 1m 55s
4000:	learn: 0.3094288	test: 0.3100905	test1: 0.2975152	best: 0.2975152 (4000)	total: 3m 49s	remaining: 57.3s
4999:	learn: 0.2918380	test: 0.2924789	test1: 0.2802986	best: 0.2802986 (4999)	total: 4m 46s	remaining: 0us

bestTest = 0.280298633
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738732	test1: 0.6742141	best: 0.6742141 (0)	total: 93.9ms	remaining: 7m 49s
1000:	learn: 0.3744377	test: 0.3737153	test1: 0.3874420	best: 0.3874378 (999)	total: 56s	remaining: 3m 43s
2000:	learn: 0.3495167	test: 0.3487594	test1: 0.3631494	best: 0.3631494 (2000)	total: 1m 51s	remaining: 2m 47s
3000:	learn: 0.3290208	test: 0.3282840	test1: 0.3422851	best: 0.3422851 (3000)	total: 2m 47s	remaining: 1m 51s
4000:	learn: 0.3094288	test: 0.3087540	test1: 0.3215760	best: 0.3215760 (4000)	total: 3m 43s	remaining: 55.9s
4999:	learn: 0.2918380	test: 0.2911786	test1: 0.3037082	best: 0.3037082 (4999)	total: 4m 43s	remaining: 0us

bestTest = 0.303708162
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738894	test1: 0.6739231	best: 0.6739231 (0)	total: 74.7ms	remaining: 6m 13s
1000:	learn: 0.3744377	test: 0.3742768	test1: 0.3773340	best: 0.3773340 (1000)	total: 57.5s	remaining: 3m 49s
2000:	learn: 0.3495167	test: 0.3493917	test1: 0.3517659	best: 0.3517659 (2000)	total: 1m 53s	remaining: 2m 49s
3000:	learn: 0.3290208	test: 0.3288920	test1: 0.3313408	best: 0.3313408 (3000)	total: 2m 49s	remaining: 1m 52s
4000:	learn: 0.3094288	test: 0.3093084	test1: 0.3115946	best: 0.3115946 (4000)	total: 3m 45s	remaining: 56.3s
4999:	learn: 0.2918380	test: 0.2917329	test1: 0.2937290	best: 0.2937290 (4999)	total: 4m 43s	remaining: 0us

bestTest = 0.2937289743
bestIteration = 4999



0:	learn: 0.6738912	test: 0.6738951	test1: 0.6738197	best: 0.6738197 (0)	total: 106ms	remaining: 8m 48s
1000:	learn: 0.3744377	test: 0.3746244	test1: 0.3710768	best: 0.3710768 (1000)	total: 1m 5s	remaining: 4m 21s
2000:	learn: 0.3495167	test: 0.3496021	test1: 0.3479793	best: 0.3479793 (2000)	total: 2m 3s	remaining: 3m 5s
3000:	learn: 0.3290208	test: 0.3290287	test1: 0.3288802	best: 0.3288802 (3000)	total: 3m	remaining: 2m
4000:	learn: 0.3094288	test: 0.3093977	test1: 0.3099871	best: 0.3099871 (4000)	total: 3m 59s	remaining: 59.9s
4999:	learn: 0.2918380	test: 0.2917985	test1: 0.2925485	best: 0.2925485 (4999)	total: 4m 57s	remaining: 0us

bestTest = 0.292548456
bestIteration = 4999



In [16]:
np.mean(test_scores)

4.606241903365218

In [18]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size = 0.2, random_state = 4)

In [19]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
cat = CatBoostClassifier()
cat.fit(Xtrain, ytrain)
pred_y = cat.predict_proba(Xtest)
print(roc_auc_score(ytest, pred_y[:,1]))
scores = cross_val_score(cat,pred_y, y_test,scoring='roc_auc')
scores

Learning rate set to 0.052243
0:	learn: 0.6598930	total: 92.4ms	remaining: 1m 32s
1:	learn: 0.6324345	total: 142ms	remaining: 1m 10s
2:	learn: 0.6074839	total: 202ms	remaining: 1m 7s
3:	learn: 0.5870693	total: 258ms	remaining: 1m 4s
4:	learn: 0.5688421	total: 309ms	remaining: 1m 1s
5:	learn: 0.5544166	total: 358ms	remaining: 59.3s
6:	learn: 0.5405587	total: 420ms	remaining: 59.6s
7:	learn: 0.5277297	total: 474ms	remaining: 58.8s
8:	learn: 0.5155060	total: 528ms	remaining: 58.1s
9:	learn: 0.5062823	total: 577ms	remaining: 57.1s
10:	learn: 0.4976860	total: 637ms	remaining: 57.3s
11:	learn: 0.4898507	total: 701ms	remaining: 57.7s
12:	learn: 0.4830429	total: 756ms	remaining: 57.4s
13:	learn: 0.4770526	total: 809ms	remaining: 57s
14:	learn: 0.4717266	total: 869ms	remaining: 57s
15:	learn: 0.4663968	total: 931ms	remaining: 57.3s
16:	learn: 0.4622555	total: 995ms	remaining: 57.5s
17:	learn: 0.4580553	total: 1.06s	remaining: 58.1s
18:	learn: 0.4546228	total: 1.13s	remaining: 58.1s
19:	learn: 0

ValueError: Found input variables with inconsistent numbers of samples: [11200, 2947]

In [20]:
np.mean(scores)

0.8907858248708691

In [25]:
prediction = test_scores.predict_proba(test_trans)[:,1]
pd.DataFrame({'Applicant_ID':tet,'default_status':prediction}).to_csv(r'C:\Users\King alagbe\Documents\pgymmm.csv', index=False)
print('Donee')

AttributeError: 'numpy.ndarray' object has no attribute 'predict_proba'

In [22]:
a = pred_y*0.8
b = test_scores*0.2

In [23]:
final = a+b

ValueError: operands could not be broadcast together with shapes (11200,2) (24000,) 

In [ ]:
sample['default_status'] = final
sample.to_csv('r"C:\Users\King alagbe\Documents\myyg.csv' , index = False)